In [54]:
#pip install --upgrade ipykernel

In [55]:
#!pip install pyLDAvis

In [2]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

/usr/local/lib/python3.10/dist-packages/joblib/backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa
2023-05-07 13:12:29.463395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 13:12:32.171087: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
with open('drive/MyDrive/Diplom/test.csv') as f:
    test = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/train.csv') as f:
    train = pd.read_csv(f,sep='\t')

In [35]:
def bootstrap(data, freq):
    freq = freq.set_index('Y')

    def sampleClass(classgroup):
        cls = classgroup['Y'].iloc[0]
        nDesired = freq.nostoextract[cls]
        nRows = len(classgroup)

        nSamples = min(nRows, nDesired)
        return classgroup.sample(nSamples)

    samples = data.groupby('Y').apply(sampleClass)

    samples.index = range(len(samples))

    return samples
    
freq = pd.DataFrame({'Y':[0, 1, 2],
                     'nostoextract':[1000, 1000, 1000]})
#6552

train = bootstrap(train, freq)

In [36]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['say', 'get'])
#'come','order','try','go','get','make','drink','plate','dish','restaurant','place',
#                 'would','really','like','great','service','came','got'])

In [37]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

words_tr = list(sent_to_words(train.body))
words_te = list(sent_to_words(test.body))

In [38]:
words_tr[21][:10]

['title',
 'stewie',
 'griffin',
 'cool',
 'whip',
 'http',
 'youtu',
 'be',
 'corpnpzkuom',
 'views']

In [39]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

words_tr = remove_stopwords(words_tr)

In [40]:
def bigrams(words, bi_min=5, tri_min=5):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(words_tr)

In [41]:
nlp = spacy.load("en_core_web_sm")

In [42]:
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [43]:
trigrams_tr = [trigram_tr[bigram_tr[review]] for review in words_tr]

In [44]:
lemma_lg = lemmatization(trigrams_tr)

In [45]:
with open('lemma_lg.pkl', 'wb') as f:
    pickle.dump(lemma_lg, f)

In [46]:
id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

In [47]:
with open('corpus_lg.pkl', 'wb') as f:
    pickle.dump(corpus_lg, f)

In [48]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg)

In [49]:
len(hdp.print_topics())

20

In [53]:
hdp.print_topics(num_topics=20)

[(0,
  '0.158*go + 0.154*say + 0.153*comment + 0.119*subreddit + 0.085*make + 0.070*think + 0.057*post + 0.056*com + 0.050*link + 0.033*use'),
 (1,
  '0.228*think + 0.158*post + 0.118*link + 0.083*com + 0.078*subreddit + 0.076*make + 0.055*use + 0.040*time + 0.036*know + 0.035*want'),
 (2,
  '0.274*want + 0.171*time + 0.146*post + 0.086*make + 0.057*think + 0.053*use + 0.044*go + 0.030*good + 0.028*com + 0.025*people'),
 (3,
  '0.332*think + 0.097*post + 0.086*people + 0.081*say + 0.079*make + 0.059*link + 0.053*subreddit + 0.040*comment + 0.039*know + 0.038*want'),
 (4,
  '0.162*see + 0.150*use + 0.115*good + 0.104*comment + 0.098*want + 0.083*com + 0.055*know + 0.050*people + 0.049*post + 0.044*think'),
 (5,
  '0.332*time + 0.127*make + 0.107*good + 0.087*want + 0.069*link + 0.069*go + 0.046*say + 0.043*post + 0.029*think + 0.024*people'),
 (6,
  '0.163*want + 0.158*post + 0.108*say + 0.090*subreddit + 0.077*people + 0.076*com + 0.074*think + 0.064*comment + 0.038*time + 0.036*link')